In [13]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

# Step 1: Split the data into training and test sets
train_size = int(len(clean_data) * 0.8)
train_data, test_data = clean_data[:train_size], clean_data[train_size:]

# Step 2: Scaling the Data (features and target)
scaler = MinMaxScaler(feature_range=(0, 1))  # To scale features
target_scaler = MinMaxScaler(feature_range=(0, 1))  # To scale target (Close*)

# Scale the features (Close*, Volume, EMA_14, RSI_14, ATR_14)
scaled_train = scaler.fit_transform(train_data[['Close*', 'Volume', 'EMA_10']])
scaled_test = scaler.transform(test_data[['Close*', 'Volume', 'EMA_10']])

# Scale the target (Close* price) separately
target_train = target_scaler.fit_transform(train_data[['Close*']])
target_test = target_scaler.transform(test_data[['Close*']])

# Step 3: Creating Sequences (timesteps)
def create_sequences(data, target, timesteps=10):
    X, y = [], []
    for i in range(len(data) - timesteps):
        X.append(data[i:i+timesteps])  # Select the past 'timesteps' rows of features
        y.append(target[i+timesteps])  # Select the corresponding target value
    return np.array(X), np.array(y)

timesteps = 10  # You can adjust this as needed
X_train, y_train = create_sequences(scaled_train, target_train, timesteps)
X_test, y_test = create_sequences(scaled_test, target_test, timesteps)

# Step 4: Build the LSTM Model (Simplified)
model = Sequential([
    LSTM(units=64, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    Dense(units=32, activation='relu'),
    Dense(units=1)  # Output layer for the target variable (Close price)
])

# Compile the model using RMSprop optimizer
model.compile(optimizer='rmsprop', loss='mean_squared_error')

# Step 5: Train the Model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Step 6: Make Predictions and Inverse Scaling
# Predict on test data
predicted_scaled = model.predict(X_test)

# Inverse scale the predictions and the true test values
predicted = target_scaler.inverse_transform(predicted_scaled)
y_test_true = target_scaler.inverse_transform(y_test)

# Check the first few predicted vs actual values
print("Predicted:", predicted[:5].flatten())
print("Actual:", y_test_true[:5].flatten())

In [27]:
# Calculate Metrics (MAE, RMSE, R²)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test_true, predicted)
rmse = np.sqrt(mean_squared_error(y_test_true, predicted))
r2 = r2_score(y_test_true, predicted)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Absolute Error: 247.53
Root Mean Squared Error: 339.80
R-squared: 0.84
